# Prep downloaded census data for plotting

Requires having run `Census-Download.ipynb` or otherwise acquired input files:
* `census_geo.*` (`shp` extension is read in here; `shx`, `cpg`, `dbf`, `prj` also produced by previous notebook)
* `census_long.csv`

Outputs several cleaned CSV files for use in plotting notebooks:
* `advanced_degree_by_age.csv`
* `counties.csv`
* `education_by_age_female.csv`
* `education_by_age_male.csv`
* `education_by_age.csv`

## Read in long format data

In [1]:
import pandas as pd

In [2]:
census_df = pd.read_csv('census_long.csv')
census_df.head()

,GEOID,estimate,sex,age,education
0,6075,203,Male,18 to 24 years,Less than 9th grade
1,6081,766,Male,18 to 24 years,Less than 9th grade
2,6001,820,Male,18 to 24 years,Less than 9th grade
3,6039,237,Male,18 to 24 years,Less than 9th grade
4,6107,491,Male,18 to 24 years,Less than 9th grade


## Education by age

I want to reduce the number of categories in education to provide a reasonable number for various plotting scenarios.
Also save data split by sex, and a separate simple CSV with only data for advanced degrees

### Combine education categories

In [3]:
census_df['education'].unique()

array(['Less than 9th grade', '9th to 12th grade, no diploma',
       'High school graduate (includes equivalency)',
       'Some college, no degree', "Associate's degree",
       "Bachelor's degree", 'Graduate or professional degree'],
      dtype=object)

In [4]:
education_map = {
    'Less than 9th grade': 'No diploma',
    '9th to 12th grade, no diploma': 'No diploma',
    'High school graduate (includes equivalency)': 'High School diploma',
    'Some college, no degree': 'Some college',
    "Associate's degree": 'Some college',
    "Bachelor's degree": "Bachelor's degree",
    'Graduate or professional degree': "Advanced degree"
}

In [5]:
ed5_df = pd.DataFrame(census_df, copy=True)
ed5_df['education'] = ed5_df['education'].replace(education_map)

### Group by education level and age

In [6]:
ed_by_age = ed5_df \
  .groupby(['age', 'education']) \
  .agg({'estimate': sum}) \
  .reset_index()
age_totals = ed_by_age.groupby(['age'])['estimate'].transform('sum')
ed_by_age['pct'] = 100 * ed_by_age['estimate'] / age_totals
ed_by_age_wide = ed_by_age.pivot(index='age', columns='education', values='pct')
ed_by_age_wide.head()

education,Advanced degree,Bachelor's degree,High School diploma,No diploma,Some college
age,,,,,
18 to 24 years,0.738296,9.987076,30.384257,11.036496,47.853875
25 to 34 years,10.713565,25.769991,21.471418,10.963738,31.081288
35 to 44 years,14.623810,22.270633,19.605535,16.824875,26.675147
45 to 64 years,12.504740,19.669680,20.241137,18.421095,29.163347
65 years and over,13.672428,17.787053,20.696619,19.705613,28.138288


In [7]:
ed_by_age_wide.to_csv('education_by_age.csv')

### Save out data for advanced degrees only

In [8]:
adv_by_age = ed_by_age_wide.rename({'Advanced degree': 'percent'}, axis=1)[['percent']]
adv_by_age.to_csv('advanced_degree_by_age.csv')

### Save out data split by sex

In [9]:
for sex in ed5_df['sex'].unique():
    ed_by_age_by_sex = ed5_df[ed5_df['sex'] == sex] \
        .groupby(['age', 'education']) \
        .agg({'estimate': sum}) \
        .reset_index()
    age_sex_totals = ed_by_age_by_sex.groupby(['age'])['estimate'].transform('sum')
    ed_by_age_by_sex['pct'] = 100 * ed_by_age_by_sex['estimate'] / age_sex_totals
    ed_by_age_by_sex_wide = ed_by_age_by_sex.pivot(index='age', columns='education', values='pct')
    ed_by_age_by_sex_wide.to_csv(f'education_by_age_{sex.lower()}.csv')

## Per-county data

Derive the county area from the geodata.
Derive the the total population and Male to Female ratio from the long format data.
Save out to plain csv file.

### Read in geodata

In [10]:
import geopandas

In [11]:
census_df_geo = geopandas.read_file("census_geo.shp")

### Calculate area

In [12]:
# See https://geopandas.org/en/stable/docs/user_guide/projections.html
# and https://gis.stackexchange.com/questions/329349/calculating-the-area-by-square-feet-with-geopandas
# Seems this data is already in epsg 3857
# https://spatialreference.org/ref/epsg/2225/
# Convert to square miles
census_df_geo['area'] = census_df_geo['geometry'].to_crs(epsg=2225).area * 3.58701 * 10**-8

In [13]:
census_df_geo.head()

,GEOID,NAME,geometry,area
0,6075,"San Francisco County, California","MULTIPOLYGON (((-13649137.130 4553355.950, -13...",232.497093
1,6081,"San Mateo County, California","POLYGON ((-13646334.440 4521323.060, -13646298...",743.365017
2,6001,"Alameda County, California","POLYGON ((-13612245.300 4538149.390, -13612345...",823.373527
3,6039,"Madera County, California","POLYGON ((-13370181.620 4462431.440, -13370113...",2161.238868
4,6107,"Tulare County, California","POLYGON ((-13225170.620 4271811.660, -13225307...",4868.434787


### Get total population per county

In [14]:
county_pop = census_df.groupby('GEOID').sum('estimate')
county_pop.head()

,estimate
GEOID,
6001,1312727
6003,838
6005,32582
6007,180378
6009,37703


### Get Male to Female ratio per county

In [15]:
pop_ratio = census_df \
    .groupby(['GEOID', 'sex']) \
    .sum('estimate') \
    .unstack() \
    .droplevel(0, axis=1) \
    .apply(lambda x: x['Male']/x['Female'], axis=1) \
    .rename('ratio')
pop_ratio.head()

GEOID
6001    0.947308
6003    1.154242
6005    1.188621
6007    0.959693
6009    0.982803
Name: ratio, dtype: float64

### Join county data, drop geometry, and save as plain csv

In [16]:
census_df_geo = census_df_geo.merge(county_pop, left_on='GEOID', right_index=True)
census_df_geo = census_df_geo.merge(pop_ratio, left_on='GEOID', right_index=True)
census_df_geo['density'] = census_df_geo['estimate'] / census_df_geo['area']
census_df_geo.head()

,GEOID,NAME,geometry,area,estimate,ratio,density
0,6075,"San Francisco County, California","MULTIPOLYGON (((-13649137.130 4553355.950, -13...",232.497093,757415,1.040857,3257.739650
1,6081,"San Mateo County, California","POLYGON ((-13646334.440 4521323.060, -13646298...",743.365017,607919,0.958609,817.793393
2,6001,"Alameda County, California","POLYGON ((-13612245.300 4538149.390, -13612345...",823.373527,1312727,0.947308,1594.327432
3,6039,"Madera County, California","POLYGON ((-13370181.620 4462431.440, -13370113...",2161.238868,112580,0.897362,52.090494
4,6107,"Tulare County, California","POLYGON ((-13225170.620 4271811.660, -13225307...",4868.434787,318740,0.979924,65.470734


In [17]:
census_df_geo.drop(['geometry'], axis=1).to_csv('counties.csv', index=False)